pca_tfidf_100features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings 
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/tfidf_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.852374,-0.444792,-0.138596,0.055378,4.798915,1.906957,-2.255331,3.778860,-2.671798,...,0.833239,0.745641,3.160825,-3.070708,1.980997,-0.612865,-1.779598,-3.798390,0.954971,-2.649764
1,1,-0.781754,-0.511917,-0.134986,-0.079779,-1.042678,-0.178863,0.113901,-0.200498,-0.119035,...,0.262187,0.065066,0.066271,0.048706,0.103881,0.111761,-0.197160,-0.055469,-0.071116,-0.152900
2,2,-0.846932,-0.582200,-0.151572,-0.112744,-1.182377,-0.226704,0.231331,-0.252564,-0.139676,...,-0.634184,-0.003201,1.698942,-1.463106,1.412316,0.763168,0.651317,1.613013,0.737140,0.406593
3,3,-0.842072,-0.576920,-0.150197,-0.109419,-0.546407,-0.413315,0.143754,-0.226513,-0.086391,...,-0.325150,0.188824,-0.293152,0.282037,0.584001,-0.124541,0.319068,-0.219725,0.011056,-0.496740
4,4,-0.832973,-0.528153,-0.141789,-0.104655,-0.860068,-0.231366,0.079792,-0.221413,-0.057949,...,-0.058929,0.054988,0.365213,-0.076371,0.816210,0.030904,0.815233,-0.085470,0.033599,-0.700580


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.852374,-0.444792,-0.138596,0.055378,4.798915,1.906957,-2.255331,3.778860,-2.671798,1.370570,...,0.833239,0.745641,3.160825,-3.070708,1.980997,-0.612865,-1.779598,-3.798390,0.954971,-2.649764
1,-0.781754,-0.511917,-0.134986,-0.079779,-1.042678,-0.178863,0.113901,-0.200498,-0.119035,0.077121,...,0.262187,0.065066,0.066271,0.048706,0.103881,0.111761,-0.197160,-0.055469,-0.071116,-0.152900
2,-0.846932,-0.582200,-0.151572,-0.112744,-1.182377,-0.226704,0.231331,-0.252564,-0.139676,0.061359,...,-0.634184,-0.003201,1.698942,-1.463106,1.412316,0.763168,0.651317,1.613013,0.737140,0.406593
3,-0.842072,-0.576920,-0.150197,-0.109419,-0.546407,-0.413315,0.143754,-0.226513,-0.086391,-0.112136,...,-0.325150,0.188824,-0.293152,0.282037,0.584001,-0.124541,0.319068,-0.219725,0.011056,-0.496740
4,-0.832973,-0.528153,-0.141789,-0.104655,-0.860068,-0.231366,0.079792,-0.221413,-0.057949,0.023470,...,-0.058929,0.054988,0.365213,-0.076371,0.816210,0.030904,0.815233,-0.085470,0.033599,-0.700580


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train) 
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.36013635065302285


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.490984,0.490984,0.584726,0.479133,0.314885,0.314885,0.352805,0.29391


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9917    0.9945    0.9931       361
           2     0.9982    0.9946    0.9964       553
           3     0.9967    1.0000    0.9984       306

    accuracy                         0.9959      1220
   macro avg     0.9955    0.9963    0.9959      1220
weighted avg     0.9959    0.9959    0.9959      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2571    0.1146    0.1586       157
           2     0.4548    0.7637    0.5701       237
           3     0.3214    0.1385    0.1935       130

    accuracy                         0.4141       524
   macro avg     0.3444    0.3389    0.3074       524
weighted avg     0.3625    0.4141    0.3534       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.490984,0.490984,0.584726,0.479133,0.314885,0.314885,0.352805,0.293910
1,RandomForest,0.995902,0.995902,0.995908,0.995902,0.414122,0.414122,0.362478,0.353375


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5680    0.3934    0.4648       361
           2     0.5455    0.7703    0.6387       553
           3     0.5767    0.3562    0.4404       306

    accuracy                         0.5549      1220
   macro avg     0.5634    0.5066    0.5146      1220
weighted avg     0.5600    0.5549    0.5375      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2929    0.1847    0.2266       157
           2     0.4353    0.6245    0.5130       237
           3     0.2000    0.1308    0.1581       130

    accuracy                         0.3702       524
   macro avg     0.3094    0.3133    0.2992       524
weighted avg     0.3343    0.3702    0.3391       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.536066,0.536066,0.540612,0.505761,0.402672,0.402672,0.336541,0.344690
1,RandomForest,0.995902,0.995902,0.995908,0.995904,0.412214,0.412214,0.365024,0.360350
2,Adaboost,0.554918,0.554918,0.559968,0.537501,0.370229,0.370229,0.334265,0.339139


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9649    0.9141    0.9388       361
           2     0.8980    0.9873    0.9406       553
           3     0.9963    0.8791    0.9340       306

    accuracy                         0.9385      1220
   macro avg     0.9531    0.9269    0.9378      1220
weighted avg     0.9425    0.9385    0.9384      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2159    0.1210    0.1551       157
           2     0.4466    0.6878    0.5415       237
           3     0.1690    0.0923    0.1194       130

    accuracy                         0.3702       524
   macro avg     0.2772    0.3004    0.2720       524
weighted avg     0.3086    0.3702    0.3210       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.536066,0.536066,0.540612,0.505761,0.402672,0.402672,0.336541,0.344690
1,RandomForest,0.995902,0.995902,0.995908,0.995904,0.412214,0.412214,0.365024,0.360350
2,Adaboost,0.554918,0.554918,0.559968,0.537501,0.370229,0.370229,0.334265,0.339139
3,GBM,0.938525,0.938525,0.942466,0.938415,0.370229,0.370229,0.308603,0.321022


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5694    0.3296    0.4175       361
           2     0.6614    0.3816    0.4839       553
           3     0.3410    0.7712    0.4729       306

    accuracy                         0.4639      1220
   macro avg     0.5240    0.4941    0.4581      1220
weighted avg     0.5538    0.4639    0.4615      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2198    0.1274    0.1613       157
           2     0.4420    0.2574    0.3253       237
           3     0.2305    0.5231    0.3200       130

    accuracy                         0.2844       524
   macro avg     0.2974    0.3026    0.2689       524
weighted avg     0.3230    0.2844    0.2749       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.490984,0.490984,0.584726,0.479133,0.314885,0.314885,0.352805,0.293910
1,RandomForest,0.995902,0.995902,0.995908,0.995902,0.414122,0.414122,0.362478,0.353375
2,SVM,0.463934,0.463934,0.553837,0.461538,0.284351,0.284351,0.322963,0.274860


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4929    0.4792    0.4860       361
           2     0.5576    0.7179    0.6277       553
           3     0.5287    0.2712    0.3585       306

    accuracy                         0.5352      1220
   macro avg     0.5264    0.4895    0.4907      1220
weighted avg     0.5312    0.5352    0.5182      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2958    0.2675    0.2809       157
           2     0.4542    0.5865    0.5120       237
           3     0.2632    0.1538    0.1942       130

    accuracy                         0.3836       524
   macro avg     0.3377    0.3360    0.3290       524
weighted avg     0.3594    0.3836    0.3639       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.536066,0.536066,0.540612,0.505761,0.402672,0.402672,0.336541,0.344690
1,RandomForest,0.995902,0.995902,0.995908,0.995904,0.412214,0.412214,0.365024,0.360350
2,Adaboost,0.554918,0.554918,0.559968,0.537501,0.370229,0.370229,0.334265,0.339139
3,GBM,0.938525,0.938525,0.942466,0.938415,0.370229,0.370229,0.308603,0.321022
4,SVM,0.527869,0.527869,0.654932,0.444039,0.433206,0.433206,0.368354,0.324791
5,KNN,0.535246,0.535246,0.531183,0.518230,0.383588,0.383588,0.359359,0.363906


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4034    0.1330    0.2000       361
           2     0.4868    0.0669    0.1176       553
           3     0.2663    0.8922    0.4102       306

    accuracy                         0.2934      1220
   macro avg     0.3855    0.3640    0.2426      1220
weighted avg     0.4068    0.2934    0.2154      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2708    0.0828    0.1268       157
           2     0.4737    0.0759    0.1309       237
           3     0.2420    0.8154    0.3732       130

    accuracy                         0.2615       524
   macro avg     0.3288    0.3247    0.2103       524
weighted avg     0.3554    0.2615    0.1898       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.536066,0.536066,0.540612,0.505761,0.402672,0.402672,0.336541,0.344690
1,RandomForest,0.995902,0.995902,0.995908,0.995904,0.412214,0.412214,0.365024,0.360350
2,Adaboost,0.554918,0.554918,0.559968,0.537501,0.370229,0.370229,0.334265,0.339139
3,GBM,0.938525,0.938525,0.942466,0.938415,0.370229,0.370229,0.308603,0.321022
4,SVM,0.527869,0.527869,0.654932,0.444039,0.433206,0.433206,0.368354,0.324791
5,KNN,0.535246,0.535246,0.531183,0.518230,0.383588,0.383588,0.359359,0.363906
6,GNB,0.293443,0.293443,0.406834,0.215398,0.261450,0.261450,0.355430,0.189807
